## 神经网络示例
使用TensorFlow构建一个2层的全连接层神经网络(多层感知机)  

- 作者: Aymeric Damien
- 代码: https://github.com/aymericdamien/TensorFlow-Examples/

#### 神经网络概览

<img src="http://cs231n.github.io/assets/nn1/neural_net2.jpeg" alt="nn" style="width: 400px;"/>

#### MNIST数据集简介
  
该例使用了MNIST手写数字数据集。MNIST数据集包含60000个实例，其中50000作为训练集，10000作为测试集。数字的大小已经被标准化和中心化到了固定的(0,1)区间(28\*28像素)。为了简便起见，每个图像矩阵被平铺，并转换为一个1维的numpy矩阵，其中包含784个特征点(28\*28)  
  
![MNIST 数据集](http://neuralnetworksanddeeplearning.com/images/mnist_100_digits.png)  
  
更多细节：http://yann.lecun.com/exdb/mnist/

In [2]:
from __future__ import print_function

#### 1. 导入MNIST数据集

In [3]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('data/',one_hot = True) #当前目录下创建一个data文件夹即可

import tensorflow as tf

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz


#### 2. 定义参数

In [13]:
#超参数
learning_rate = 0.1
num_steps = 500
batch_size = 128
display_step = 100

##网络模型参数
n_hidden_1 = 256     #第一层的神经元个数
n_hidden_2 = 256     #第二层的神经元格式
num_input = 784      #MNIST的输入数据(28*28像素)
num_classes = 10     #MNIST类别(0-9个数字)

#TensorFlow的Graph输入
X = tf.placeholder('float',[None,num_input])
Y = tf.placeholder('float',[None,num_classes])


#保存weights和bias
weights = {
    'h1': tf.Variable(tf.random_normal([num_input,n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1,n_hidden_2])),
    'out':tf.Variable(tf.random_normal([n_hidden_2,num_classes])),
}

biases = {
    'b1':tf.Variable(tf.random_normal([n_hidden_1])),
    'b2':tf.Variable(tf.random_normal([n_hidden_2])),
    'out':tf.Variable(tf.random_normal([num_classes])),
}

#### 3. 创建模型


In [14]:
def neural_net(x):
    # 含有256个神经元的隐全连接层
    layer_1 = tf.add(tf.matmul(x,weights['h1']),biases['b1'])
    # 含有256个神经元的隐全连接层
    layer_2 = tf.add(tf.matmul(layer_1,weights['h2']),biases['b2'])
    # 输出全连接层，每个类有一个神经元
    out_layer = tf.matmul(layer_2,weights['out']) + biases['out']
    return out_layer

logits = neural_net(X)

#定义loss和optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits= logits,labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate)
train_op = optimizer.minimize(loss_op)

#模型评估（使用了test logits，没有设置dropout）
correct_pred = tf.equal(tf.argmax(logits,1),tf.arg_max(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32))

#初始化变量
init = tf.global_variables_initializer()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



Instructions for updating:
Use `argmax` instead


Instructions for updating:
Use `argmax` instead


#### 4. 训练模型

In [18]:
with tf.Session() as sess:
    #初始化
    sess.run(init)
    
    for step in range(1,num_steps+1):
        batch_x,batch_y = mnist.train.next_batch(batch_size)
        #运行optimizer（后向传播）
        sess.run(train_op,feed_dict={X:batch_x,Y:batch_y})
        if step % display_step == 0 or step == 1:
            #计算loss和accuracy
            loss,acc = sess.run([loss_op,accuracy],feed_dict = {X:batch_x,
                                                               Y:batch_y})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))
            
            
    print ('训练结束！')
    
    #模型预测
    
    print ('Testing Accuracy:',sess.run(accuracy,feed_dict={X:mnist.test.images,
                                                           Y:mnist.test.labels}))
    

Step 1, Minibatch Loss= 12121.3994, Training Accuracy= 0.312
Step 100, Minibatch Loss= 239.1489, Training Accuracy= 0.883
Step 200, Minibatch Loss= 224.4041, Training Accuracy= 0.836
Step 300, Minibatch Loss= 65.8013, Training Accuracy= 0.859
Step 400, Minibatch Loss= 68.7485, Training Accuracy= 0.883
Step 500, Minibatch Loss= 86.7834, Training Accuracy= 0.836
训练结束！
Testing Accuracy: 0.8549
